# 1. Input-output operations

*chython.files* subpackage contains file readers and writers classes.

## 1.1. MDL RDF reader

**RDFRead** class can be used for RDF files reading.
Instance of this class is file-like object which support **iteration**, has a method **read()** for parsing all data and **context manager**.

### 1.1.1. Read file from disk

In [ ]:
from chython.files import * # import all available readers and writers

with RDFRead('example.rdf') as f:
    first = next(f)  # get first reaction using generator
    data = f.read()  # read remaining reactions to list of ReactionContainers

data = []
with RDFRead('example.rdf') as f:
    for r in f:  # looping is supported. Useful for large files.
        data.append(r)

#### OOP-stype Pathlib supported

In [ ]:
from pathlib import Path

with RDFRead(Path('example.rdf')) as r: # OOP style call
    r = next(r)

#### opened files supported
RDF file should be opened in text mode

In [ ]:
with open('example.rdf') as f, RDFRead(f) as r:
    r = next(r) # OOP style application

### 1.1.2. Transparent loading from archives and network
Readers designed transparently support any type of data sources. 

Data sources should be file-like objects.

In [ ]:
from requests import get
from io import StringIO

# get function return requested URL which has attribute text. 
# in example this text is whole RDF stored in single string.
# RDFread does not support parsing of strings, but one can emulate files with data 
# instead of strings by using io.StringIO
with StringIO(get('https://github.com/chython/chython/raw/master/doc/tutorial/example.rdf').text) as f, RDFRead(f) as r:
    r = next(r)

# python support gzipped data. This example shows how to work with compressed 
# data directly without decompressing them to disk.
from gzip import open as gzip_open
with gzip_open('example.rdf.gz', 'rt') as f, RDFRead(f) as r:
    r = next(r)

# zip-files also supported out of the box 
# zipped files can be opened only in binary mode. io.TextIOWrapper can be used for transparent decoding them into text
from zipfile import ZipFile
from io import TextIOWrapper
with ZipFile('example.zip') as z, z.open('example.rdf') as c:
    with TextIOWrapper(c) as f, RDFRead(f) as r:
        r = next(r)

# tar-file reading example
from tarfile import open as tar_open
from io import TextIOWrapper
with tar_open('example.tar.gz') as t:
    c = t.extractfile('example.rdf')
    with TextIOWrapper(c) as f, RDFRead(f) as r:
        r = next(r)

## 1.2. Other Readers
* SDFRead - MOL, SDF files reader (versions v2000, v3000 are supported)
* MRVRead - ChemAxon MRV files reader (lxml parser is used)
* SMILESRead - SMILES strings files reader (coho backend used). Every row should start with new SMILES
* INCHIRead - INCHI strings files reader (INCHI trust backend used). Every row should start with new InChI
* XYZRead - xyz files reader (only structures with explicit hydrogens supported)
* PDBRead - PDB files parser (only structures with explicit hydrogens supported)

All files except MRV should be opened in **text-mode**  
MRV requires binary mode `open('/path/to/data.mrv', 'rb')`

In [ ]:
with MRVRead(open('example.mrv', 'rb')) as f:
    m = next(f)

## 1.3. File writers
Export in following file formats is supported:

* RDFWrite (v2000) - molecules and reactions export in RDF format
* SDFWrite (v2000) - molecules export in SDF format
* ERDFWrite (v3000) - molecules and reactions export in RDF format
* ESDFWrite (v3000) - molecules export in SDF format
* MRVWrite - molecules and reactions export in MRV format

Writers have the same API as readers. All writers work with text-files
Writers have `write` method which accepts as argument single reaction or molecule object

In [ ]:
with RDFWrite('out.rdf') as f: # context manager supported
    for r in data:
        f.write(r)
# file out.rdf will be overriden

In [ ]:
f = RDFWrite('out.rdf') # ongoing writing into a single file
for r in data:
    f.write(r)

f.write(r)
f.close() # close file. Flushes Python writer buffers.

## 1.4. Pickle support

Chython containers fully support pickle dumping and loading.

Pickle dumps are more fast than common files and could be used as temporal storage.

In [ ]:
from pickle import loads, dumps

In [ ]:
loads(dumps(r)) # load reaction from Pickle dump

## 1.5. Chython binary format (chython pack)

Chython introduce new effective format for molecules and reactions, which combine benefits from MDL and SMILES formats.
Molecules store 2d-coordinates; tetrahedron, allene and cis-trans stereo; explicit bonds, implicit hydrogen count, atom numbers, radical mark, charge, isotope.

Size only 1.5-2 times larger than SMILES. Parsing speed is faster than pickle.
Full specification described in source code.

In [ ]:
from chython import MoleculeContainer, ReactionContainer

b = r.pack()
r = ReactionContainer.unpack(b)

# same for molecules
# MoleculeContainer.unpack(m.pack())

## 1.6. Metadata access

RDF, SDF, etc - files have metadata which stored in molecules and reactions objects

In [ ]:
r = next(RDFRead('example.rdf'))
r.meta # dictionary for molecule/reaction properties storage. For example, DTYPE/DATUM fields of RDF file are read into this dictionary

In [ ]:
r.name  # string with reaction title from RDF

In [ ]:
r.reactants[0].name # string with reactant molecule title from MOL

## 1.7. Depiction into SVG

In [ ]:
r.depict()[:100] # show only part of string

In [ ]:
r # Notebooks supported!

# 2. Signatures and duplicates selection

## 2.1. Molecule Signatures
*MoleculeContainer* has methods for unique molecule signature generation.
Signature is SMILES string with canonical atoms ordering. Order of atoms calculated by Morgan-like algorithm.

For signature generation one need to call `str` function on MoleculeContainer object.
Fixed length hash of signature could be retrieved by calling `bytes` function on molecule (correspond to SHA 512 bitstring).

Next string formatting keys supported:

a - Generate asymmetric closures.
!s - Disable stereo marks.
A - Use aromatic bonds instead aromatic atoms.
m - Set atom mapping.
r - Generate random-ordered smiles.
h - Show implicit hydrogens.
!b - Disable bonds tokens.

In [ ]:
from chython import smiles  # smiles string parser

m = smiles('c1ccccc1C=2C=CC=CC=2[C@H](O)C')
str(m) # signature
bytes(m) # cryptographic signature hash
hash(m) # runtime-dependent signature hash. See Python str hash behavior

print(m)
print(f'f string {m}')  # use signature in string formatting
print('C-style string %s' % m)
print('format method {}'.format(m))
print(f'{m:A}')
print(f'{m:h!b}') # combination supported

Molecules comparable and hashable

Comparison of MoleculeContainer is based on its signatures. Moreover, since strings in Python are hashable, MoleculeContaier also hashable.

NOTE: MoleculeContainer can be changed. This can lead to unobvious behavior of the sets and dictionaries in which these molecules were placed before the change. Avoid changing molecules (standardize, aromatize, hydrogens and atoms/bonds changes) placed inside sets and dictionaries.

In [ ]:
m != smiles('c1ccccc1')

In [ ]:
# Simplest way to exclude duplicated structures
len({m, m, smiles('c1ccccc1')}) == 2 # create set of unique molecules. Only 2 of them were different.

## 2.2. Reaction signatures
ReactionContainer have its signature. Signature is SMIRKS string in which molecules of reactants, reagents, products presented in canonical order.

API is the same as for molecules

Next extra formatting keys supported:

!c - Keep nested containers order
!C - skip cxsmiles fragments contract

In [ ]:
print(r)

In [ ]:
format(r, '!c')

# 3. Structure standardization

## 3.1. Molecules

MoleculeContainer has `standardize`, `kekule`, `thiele`, `neutralize`, `implicify_hydrogens`, `explicify_hidrogens` and `canonicalize` methods.

Method `thiele` transforms Kekule representation of rings into aromatized.
Method `standardize` applies functional group standardization rules to molecules (more than 80 rules).

Method `canonicalize` apply set of methods: `neutralize`, `standardize`, `kekule`, `implicify_hydrogens`, `thiele`

In [ ]:
m = smiles('c1ccccc1N(=O)=O')
m.clean2d()  # calculate 2d layout
m #  depict

In [ ]:
m.kekule()  # transform to kekule form
m

In [ ]:
m.standardize() # fix groups
m

In [ ]:
m.thiele() # transform to aromatized form
m

In [ ]:
m = smiles('[NH3+]CC(=O)[O-]')
m.clean2d()
m.neutralize() # fix zwitter-ions
m

Molecules has `explicify_hydrogens` and `implicify_hydrogens` methods to handle hydrogens.

This methods is used to add or remove hydrogens in molecule.

Note `implicify_hydrogens` working for aromatic rings only in `kekule` form. `explicify_hydrogens` for `aromatized` forms required `kekule` and optionally `thiele` procedures applyed before.

In [ ]:
print(m.explicify_hydrogens()) # return number of added hydrogens
m.clean2d()
m

In [ ]:
m.implicify_hydrogens()
m

## 3.2. Reactions
ReactionContainer has same methods as molecules. In this case they are applied to all molecules in reaction.

`explicify_hydrogen` method try to keep atom-to-atom mapping.

Reaction specific methods:

`remove_reagents` - move reactants to reagents. based on atom-to-atom mapping.
`contract_ions` - merge ions in single multicomponent molecule.

In [ ]:
r = smiles('[Na+:1].[OH-:2].[CH3:7][O:5][C:4]([CH3:3])=[O:6]>>[CH3:3][C:4]([OH:8])=[O:6]') # mapping required
r.clean2d()
r

In [ ]:
r.contract_ions()
r

In [ ]:
r.remove_reagents(keep_reagents=True)
r

In [ ]:
r.explicify_hydrogens()
r.clean2d()
r

# 4. Isomorphism

## 4.1. Molecules Isomorphism

Chython has simple substructure/structure isomorphism API.

Note, that atoms are matched in subgraph isomorphism only if they have same charge/multiplicity and isotope options.

In [ ]:
benzene = smiles('c1ccccc1')
toluene = smiles('c1ccccc1C')
# isomorphism operations
print(benzene < toluene)  # benzene is substructure of toluene
print(benzene > toluene)  # benzene is not superstructure of toluene
print(benzene <= toluene) # benzene is substructure/or same structure of toluene
print(benzene >= toluene) # benzene is not superstructure/or same structure of toluene
print(benzene < benzene) # benzene is not substructure of benzene. it's equal
print(benzene <= benzene)

Mappings of substructure or structure to structure can be returned using `substructure.get_mapping(structure)` method. Method acts as generator.

In [ ]:
next(benzene.get_mapping(toluene))

In [ ]:
for m in benzene.get_mapping(toluene, automorphism_filter=False):  # iterate over all possible substructure mappings
    print(m)

## 4.2. Queries

Queries (QueryContainer) is special objects which additionally takes into account neighbors, hybridization, hydrogen count, ring size and heteroatom neighbors count state of atoms and bond in ring state.

Queries can be generated from molecules by `substructure` method with as_query argument.

Few special arguments for controlling atom state available.

In [ ]:
m = smiles('NCC(=O)O')
carboxy = m.substructure([3, 4, 5], as_query=True, skip_neighbors_marks=False, skip_hybridizations_marks=False, skip_hydrogens_marks=False, skip_rings_sizes_marks=False)
print(carboxy)

In [ ]:
carboxy < m

In [ ]:
carboxy < smiles('NCC(=O)OC') # not acid!

### 4.2.1. Query building API

It is possible to build query and molecule objects in programming way

In [ ]:
from chython import QueryContainer, MoleculeContainer
from chython.containers.bonds import QueryBond
from chython.periodictable import ListElement

In [ ]:
q = QueryContainer() # create empty query
q.add_atom('C', neighbors=3, hybridization=2, heteroatoms=1, rings_sizes=0, hydrogens=0)
q.add_atom(ListElement(['O', 'S']), n=3) # oxygen or sulphur, with atom number 3
q.add_bond(1, 3, 2) # atoms enumerated from 1. connect first and 3rd atom by bouble bond.
print(q) # match only [tia] ketones

In [ ]:
q < smiles('CC(=O)O')

In [ ]:
q < smiles('CC(=S)C')

In [ ]:
q < smiles('CC=O')

Molecules construction API the same, except extra query attributes

In [ ]:
m = MoleculeContainer()
m.add_atom('C')
m.add_atom('C')
m.add_atom('O')
m.add_bond(1, 2, 1)
m.add_bond(2, 3, 2)
m.clean2d()
m

# 5. Reactor

Reactor works similar to ChemAxon Reactions enumeration.

Example here presents application of it to create esters from acids and alcoholes.

First we need to construct carboxy group and alcohole matcher queries. Then, ether group need to be specified. 

Atom numbers in query and patch should be mapped to each other. The same atoms should have same numbers.

In [ ]:
acid = QueryContainer()
acid.add_atom('C')
acid.add_atom('O', neighbors=1)
acid.add_atom('O')
acid.add_bond(1, 2, 1)
acid.add_bond(1, 3, 2)
print(acid)

In [ ]:
alco = QueryContainer()
alco.add_atom('C', n=4, heteroatoms=1) # set atom number manually
alco.add_atom('O', 5, neighbors=1)
alco.add_bond(4, 5, 1)
print(alco)

In [ ]:
ether = QueryContainer()
ether.add_atom('C')
ether.add_atom('O', 3)
ether.add_atom('C')
ether.add_atom('O')
ether.add_bond(1, 3, 2)
ether.add_bond(1, 5, 1)
ether.add_bond(4, 5, 1)
print(ether)

In [ ]:
from chython import Reactor
from chython.utils import grid_depict
from ipywidgets import HTML

In [ ]:
rxn = Reactor([acid, alco], [ether], delete_atoms=True, one_shot=False)
# delete atoms not presented in product query
# do multiple reactions if possible

In [ ]:
alcohols = [smiles('CO'), smiles('CCO'), smiles('CC(C)O')]
acids = [smiles('C(=O)O'), smiles('CC(=O)O'), smiles('OC(=O)C(=O)O')]
for x in alcohols + acids:
    x.clean2d()

In [ ]:
HTML(grid_depict(alcohols + acids))

In [ ]:
from itertools import product

products = []
for x in product(acids, alcohols):
    for p in rxn(x): # apply transformation on given list of reactants
        p.clean2d()
        products.append(p)
len(products)

In [ ]:
products[0]

In [ ]:
products[-3]

In [ ]:
products[-4]